## Notebook to generate collision rate violators for any RMG job while running

In [1]:
import sys, os
sys.path.append('/Users/ebubeajulu/anaconda3/envs/rmg_py3/')

!conda list rmg_py3
!which python

# packages in environment at /Users/ebubeajulu/anaconda3/envs/rmg3:
#
# Name                    Version                   Build  Channel
/Users/ebubeajulu/anaconda3/envs/rmg3/bin/python


In [2]:
import logging

In [3]:
import rmgpy

import numpy as np

from rmgpy.rmgobject import RMGObject, expand_to_dict, recursive_make_object


In [4]:
%cat chemkin/chem_annotated.inp

ELEMENTS
	H
	D /2.014/
	T /3.016/
	C
	CI /13.003/
	O
	OI /18.000/
	N
	Ne
	Ar
	He
	Si
	S
	Cl
	X /195.083/
END

SPECIES
    N2                  ! N2
    Ar                  ! Ar
    Ne                  ! Ne
    nC10H22(1)          ! nC10H22(1)
    O2(2)               ! O2(2)
    CH(3)               ! CH(3)
    C2H(4)              ! C2H(4)
    O(5)                ! O(5)
    CO(6)               ! CO(6)
    CO2(7)              ! CO2(7)
    H2O(8)              ! H2O(8)
    CH2O(9)             ! CH2O(9)
    CH4(10)             ! CH4(10)
    C2H4(11)            ! C2H4(11)
    C3H6(12)            ! C3H6(12)
    H2(13)              ! H2(13)
    H(14)               ! H(14)
    OH(15)              ! OH(15)
    He(16)              ! He(16)
    HO2(17)             ! HO2(17)
    H2O2(18)            ! H2O2(18)
    HOCO(19)            ! HOCO(19)
    HCO(20)             ! HCO(20)
    CH3(21)             ! CH3(21)
    CH2(22)             ! CH2(22)
    CH3OH(23)           ! CH3OH(23)
    CH2(S)(24)       

CH2(S)(24)+S(441)=S(863)                            8.734758e+11 0.189     0.000    

! Reaction index: Chemkin #938; RMG #1572
! Library reaction: CurranPentane
! Flux pairs: S(192), S(227); 
S(192)=S(227)                                       1.360000e+07 1.300     18.200   

! Reaction index: Chemkin #939; RMG #1589
! Library reaction: CurranPentane
! Flux pairs: S(227), OH(15); S(227), CH2O(9); S(227), C3H6(12); 
S(227)=>OH(15)+CH2O(9)+C3H6(12)                     1.230000e+09 1.300     24.900   

! Reaction index: Chemkin #940; RMG #858
! Library reaction: CurranPentane
! Flux pairs: O2(2), S(104); NC3H7(101), S(104); 
O2(2)+NC3H7(101)=S(104)                             1.000e+00 0.000     0.000    
    PLOG/ 0.010000  9.200e+08 0.405     -4.399   /
    PLOG/ 0.100000  1.450e+14 -0.984    -1.711   /
    PLOG/ 1.000000  2.090e+13 -0.499    -0.938   /
    PLOG/ 10.000000 1.150e+20 -2.420    2.451    /
    PLOG/ 100.000000 2.070e+16 -1.300    0.803    /

! Reaction index: Chemkin #94

! Euclidian distance = 0
! family: R_Addition_MultipleBond
H(14)+S(4959)=S(1129)                               9.600000e+09 0.935     4.170    

! Reaction index: Chemkin #2212; RMG #27014
! Template reaction: Disproportionation
! Flux pairs: S(1129), S(4959); HO2(17), H2O2(18); 
! Estimated using template [O_rad;C/H2/Nd_Rrad] for rate rule [O_rad/NonDeO;C/H2/Nd_Orad]
! Euclidian distance = 2.23606797749979
! Multiplied by reaction path degeneracy 2.0
! family: Disproportionation
HO2(17)+S(1129)=H2O2(18)+S(4959)                    4.820000e+13 0.000     0.000    

! Reaction index: Chemkin #2213; RMG #27170
! Template reaction: Disproportionation
! Flux pairs: S(1129), S(4959); CH3OO(28), CH3OOH(34); 
! Estimated using template [O_rad;C/H2/Nd_Rrad] for rate rule [O_rad/NonDeO;C/H2/Nd_Orad]
! Euclidian distance = 2.23606797749979
! Multiplied by reaction path degeneracy 2.0
! family: Disproportionation
CH3OO(28)+S(1129)=CH3OOH(34)+S(4959)                4.820000e+13 0.000     0.000    


In [11]:
import rmgpy 
from rmgpy.chemkin import load_chemkin_file
species, reactions = rmgpy.chemkin.load_chemkin_file(
                    'chemkin/chem_annotated.inp','chemkin/species_dictionary.txt',transport_path='chemkin/tran.dat',
                    check_duplicates=False, use_chemkin_names=True,read_comments=True)



In [12]:
species_dict = {rmgpy.chemkin.get_species_identifier(s): s for s in species}
species_dict.keys()

dict_keys(['N2', 'Ar', 'Ne', 'nC10H22(1)', 'O2(2)', 'CH(3)', 'C2H(4)', 'O(5)', 'CO(6)', 'CO2(7)', 'H2O(8)', 'CH2O(9)', 'CH4(10)', 'C2H4(11)', 'C3H6(12)', 'H2(13)', 'H(14)', 'OH(15)', 'He(16)', 'HO2(17)', 'H2O2(18)', 'HOCO(19)', 'HCO(20)', 'CH3(21)', 'CH2(22)', 'CH3OH(23)', 'CH2(S)(24)', 'CH2OH(25)', 'CH3O(26)', 'HCOH(27)', 'CH3OO(28)', 'CH2CO(29)', 'C2H5(30)', 'C2H3(31)', 'C(32)', 'C2H2(33)', 'CH3OOH(34)', 'CH2OOH(35)', 'C2H6(36)', 'CH3CHO(37)', 'C2H5O(38)', 'C2H5O2(39)', 'CH2CHO(40)', 'H2CC(41)', 'C2H4O(42)', 'C2H5O(43)', 'cC2H4O(44)', 'C2H5O2(45)', 'C2H3O2(46)', 'CHCHO(47)', 'OCHCHO(48)', 'HCCO(49)', 'HCCOH(50)', 'CHCHOH(51)', 'C2(52)', 'C2O(53)', 'C2H6O(54)', 'C2H5O(55)', 'C2H5O3(56)', 'CH3CO(57)', 'cC2H3O(58)', 'HOCHO(59)', 'C2H3O3(60)', 'OCHCO(61)', 'C2H6O2(62)', 'C2H5O2(63)', 'C2H4O2(64)', 'HOCH2O(65)', 'OCHO(66)', 'C2H4O3(67)', 'C2H3O2(68)', 'NC3H7(101)', 'C3H5(103)', 'S(104)', 'S(123)', 'C3H4(174)', 'PC4H9(190)', 'S(192)', 'C4H8(196)', 'S(227)', 'S(228)', 'C4H6O(271)', 'C5H11(4

In [13]:

# rmgpy.chemkin.load_transport_file('chemkin/tran.dat', species_dict)
# This section of code is redundant as the main chemkin file and species 
# dictionary has already been loaded with the input file above

In [14]:
def check_collision_limit_violation(self, t_min, t_max, p_min, p_max):
        """
        Warn if a core reaction violates the collision limit rate in either the forward or reverse direction
        at the relevant extreme T/P conditions. Assuming a monotonic behaviour of the kinetics.
        Returns a list with the reaction object and the direction in which the violation was detected.
        """
        conditions = [[t_min, p_min]]
        if t_min != t_max:
            conditions.append([t_max, p_min])
        if self.kinetics.is_pressure_dependent() and p_max != p_min:
            conditions.append([t_min, p_max])
            if t_min != t_max:
                conditions.append([t_max, p_max])
        logging.debug("Checking whether reaction {0} violates the collision rate limit...".format(self))
        violator_list = []
        kf_list = []
        kr_list = []
        collision_limit_f = []
        collision_limit_r = []
        for condition in conditions:
            if len(self.reactants) >= 2:
                try:
                    collision_limit_f.append(self.calculate_coll_limit(temp=condition[0], reverse=False))
                except ValueError:
                    continue
                else:
                    kf_list.append(self.get_rate_coefficient(condition[0], condition[1]))
            if len(self.products) >= 2:
                try:
                    collision_limit_r.append(self.calculate_coll_limit(temp=condition[0], reverse=True))
                except ValueError:
                    continue
                else:
                    kr_list.append(self.generate_reverse_rate_coefficient().get_rate_coefficient(condition[0], condition[1]))
        if len(self.reactants) >= 2:
            for i, k in enumerate(kf_list):
                if k > collision_limit_f[i]:
                    ratio = k / collision_limit_f[i]
                    condition = '{0} K, {1:.1f} bar'.format(conditions[i][0], conditions[i][1] / 1e5)
                    violator_list.append([self, 'forward', ratio, condition])
        if len(self.products) >= 2:
            for i, k in enumerate(kr_list):
                if k > collision_limit_r[i]:
                    ratio = k / collision_limit_r[i]
                    condition = '{0} K, {1:.1f} bar'.format(conditions[i][0], conditions[i][1] / 1e5)
                    violator_list.append([self, 'reverse', ratio, condition])
        return violator_list

In [15]:
import numpy as np
np.linalg.LinAlgError

numpy.linalg.linalg.LinAlgError

In [ ]:
violators = []
for r in reactions:
    try:
        v = check_collision_limit_violation(r, 800,1000, 1e5, 30e5)
    except np.linalg.LinAlgError:
        print(f"LinAlgError checking {r}")
    if v:
        print(r)
        violators.extend(v)

H(14) + HO2(17) <=> H2O2(18)
LinAlgError checking O2(2) + CH2O(9) <=> HO2(17) + HCO(20)
LinAlgError checking HO2(17) + CH2O(9) <=> H2O2(18) + HCO(20)
LinAlgError checking CH2O(9) + CH3(21) <=> HCO(20) + CH4(10)
CH3(21) <=> H2(13) + CH(3)
CH3(21) <=> H(14) + CH2(22)


/Users/ebubeajulu/anaconda3/envs/rmg3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in log


CH2(22) <=> H(14) + CH(3)
LinAlgError checking OH(15) + CH(3) <=> H(14) + HCO(20)
LinAlgError checking O2(2) + CH(3) <=> O(5) + HCO(20)
LinAlgError checking CO2(7) + CH(3) <=> CO(6) + HCO(20)
LinAlgError checking HCO(20) + CH3OH(23) <=> CH2O(9) + CH2OH(25)
LinAlgError checking HCO(20) + CH3OH(23) <=> CH2O(9) + CH3O(26)
LinAlgError checking O(5) + C2H4(11) <=> HCO(20) + CH3(21)
LinAlgError checking O(5) + C2H4(11) <=> HCO(20) + CH3(21)
LinAlgError checking CH2O(9) + C2H3(31) <=> HCO(20) + C2H4(11)
C2H2(33) <=> H(14) + C2H(4)
LinAlgError checking O(5) + C2H4O(42) <=> HCO(20) + CH2OH(25)
LinAlgError checking O(5) + C2H4O(42) <=> HCO(20) + CH2OH(25)
LinAlgError checking O2(2) + C2H4O(42) => OH(15) + HCO(20) + CH2O(9)
LinAlgError checking H(14) + CH3CO(57) <=> HCO(20) + CH3(21)
LinAlgError checking H(14) + CH2CHO(40) <=> HCO(20) + CH3(21)
LinAlgError checking O(5) + CH2CHO(40) <=> HCO(20) + CH2O(9)
LinAlgError checking OH(15) + CH2CHO(40) <=> HCO(20) + CH2OH(25)
LinAlgError checking CH2(22)

In [ ]:
from rmgpy.data.kinetics import TemplateReaction, LibraryReaction
from rmgpy.rmg.pdep import PDepReaction

In [ ]:
for violator in violators:
    rxn_string = str(violator[0])
    kinetics = violator[0].kinetics
    comment = ''
    if isinstance(violator[0], TemplateReaction):
        comment = violator[0].kinetics.comment
        violator[0].kinetics.comment = ''  # the comment is printed better when outside of the object
    if isinstance(violator[0], LibraryReaction):
        comment = 'Kinetic library: {0}'.format(violator[0].library)
    if isinstance(violator[0], PDepReaction):
        comment = 'Network #{0}'.format(violator[0].network)
    direction = violator[1]
    ratio = violator[2]
    condition = violator[3]
    
    if ratio < 5:
        continue # only print the big ones!
        
    print('{0}\n{1}\n{2}\nDirection: {3}\nViolation factor: {4:.2f}\n'
                      'Violation condition: {5}\n\n'.format(rxn_string, kinetics, comment, direction,
                                                            ratio, condition))
    if isinstance(violator[0], TemplateReaction):
        # although this is the end of the run, restore the original comment
        violator[0].kinetics.comment = comment